In [1]:
!pip install bertopic[all] umap-learn hdbscan Sastrawi
!pip install sentence-transformers

In [2]:
from bertopic import BERTopic
import pandas as pd
import matplotlib.pyplot as plt
import re

Matplotlib is building the font cache; this may take a moment.


In [4]:
# Load CSV lokal
file_path = "Dataset/label_indobert_x.csv"  # pastikan path sesuai lokasi file
df = pd.read_csv(file_path)
df.head()

,comment,case_folding,cleaning,normalisasi,tokenisasi,stopword_removal,stemming,score,sentiment_label,sentiment_encoded,bert_encoded,bert_label
0,byond bsi gangguan gasih? gw topup shopay uda...,byond bsi gangguan gasih? gw topup shopay uda...,byond bsi gangguan gasih gw topup shopay udah...,byond bsi masalah tidak sih gue isi ulang shop...,"['byond', 'bsi', 'masalah', 'tidak', 'sih', 'g...","['masalah', 'tidak', 'sih', 'gue', 'isi', 'ula...","['masalah', 'tidak', 'sih', 'gue', 'isi', 'ula...",-4,Negatif,0,0,negatif
1,@blurbintang @worksfess Kalau kasus byond aku ...,@blurbintang @worksfess kalau kasus byond aku ...,kalau kasus byond aku kurang copy kak,kalau kasus byond akun kurang copy kak,"['kalau', 'kasus', 'byond', 'akun', 'kurang', ...","['akun', 'kurang', 'copy']","['akun', 'kurang', 'copy']",-1,Negatif,0,0,negatif
2,Halo min @bankbsi_id Saya melakukan pembayar s...,halo min @bankbsi_id saya melakukan pembayar s...,halo min saya melakukan pembayar shopee melal...,halo min saya melakukan pembayar shopee melalu...,"['halo', 'min', 'saya', 'melakukan', 'pembayar...","['min', 'pembayar', 'shopee', 'kali', 'statusn...","['min', 'bayar', 'shopee', 'kali', 'status', '...",3,Positif,1,1,positif
3,@asknownasaqueen @worksfess Kak nitip prtnyaan...,@asknownasaqueen @worksfess kak nitip prtnyaan...,kak nitip prtnyaan please ak mau nyoba nabun...,kak nitip prtnyaan please aku mau mencoba nabu...,"['kak', 'nitip', 'prtnyaan', 'please', 'aku', ...","['nitip', 'prtnyaan', 'please', 'mencoba', 'na...","['nitip', 'prtnyaan', 'please', 'coba', 'nabun...",-1,Negatif,0,0,negatif
4,@bankbsi_id min kok aplikasi byond by bsi gk b...,@bankbsi_id min kok aplikasi byond by bsi gk b...,min kok aplikasi byond by bsi gk bisa dibuka ...,min kok aplikasi byond by bsi tidak bisa dibuk...,"['min', 'kok', 'aplikasi', 'byond', 'by', 'bsi...","['min', 'aplikasi', 'tidak', 'dibuka', 'siang'...","['min', 'aplikasi', 'tidak', 'buka', 'siang', ...",-3,Negatif,0,0,negatif


In [5]:
df_negatif = df[df["bert_label"] == "negatif"].copy()
print("Jumlah Ulasan Negatif:", len(df_negatif))

Jumlah Ulasan Negatif: 2426


In [6]:
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# --- stopword dasar dari Sastrawi ---
factory = StopWordRemoverFactory()
base_stopwords = set(factory.get_stop_words())

# --- stopword tambahan untuk data TikTok (super lengkap) ---
extra_stopwords = {
    # kata panggilan
    "kak", "kakak", "bang", "bro", "sis", "bos", "min", "admin",

    # nama aplikasi & banking
    "bsi", "byond", "beyond", "wondr", "bni", "mbanking", "mobile", "bank",

    # kata bantu umum yang tidak bermakna topik
    "bisa", "tidak", "ga", "gak", "gk", "nggak", "ngga", "enggak",
    "iya", "kayak", "gua", "gue", "aku", "kamu", "anda", "mereka",
    "saya", "mau", "udah", "lagi", "yang", "dengan", "agar", "jadi",

    # kata noise umum TikTok
    "anjir", "anjay", "banget", "parah", "loh", "lah", "kok", "sih",
    "wkwk", "wkwkwk", "hehe", "haha", "hmm",

    # corporate / lokasi bias
    "tbk", "pt", "aceh", "indonesia",
}

# gabungkan semua stopword
all_stopwords = base_stopwords.union(extra_stopwords)

# --- CLEANING SUPER KOMPREHENSIF UNTUK BERTopic ---
def clean_bertopic(text):
    text = str(text).lower()

    # hapus URL, @mention, hashtag
    text = re.sub(r"http\S+|www\S+|@\w+|#\w+", " ", text)

    # hapus karakter selain huruf/angka/spasi
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)

    # hapus kata 1–3 huruf (mengurangi bias TikTok)
    text = re.sub(r"\b\w{1,3}\b", " ", text)

    # stopword filter
    tokens = [w for w in text.split() if w not in all_stopwords]

    return " ".join(tokens).strip()


In [7]:
df_negatif["clean_text"] = df_negatif["normalisasi"].apply(clean_bertopic)
df_negatif[["normalisasi", "clean_text"]].head()

,normalisasi,clean_text
0,byond bsi masalah tidak sih gue isi ulang shop...,masalah ulang shopay menit disuruh menunggu ma...
1,kalau kasus byond akun kurang copy kak,kalau kasus akun kurang copy
3,kak nitip prtnyaan please aku mau mencoba nabu...,nitip prtnyaan please mencoba nabung emas hrga...
4,min kok aplikasi byond by bsi tidak bisa dibuk...,aplikasi dibuka siang makan
5,baik terima kasih omong-omong kartu atm bsi sa...,baik terima kasih omong omong kartu hilang buk...


In [8]:
# 5. EMBEDDING MODEL TERBAIK
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# 6. BERTopic MODEL

topic_model = BERTopic(
    embedding_model=embedding_model,
    language="indonesian",
    min_topic_size=30,     # cluster minimal agar topik stabil
    nr_topics="auto"       # otomatis menentukan jumlah topik
)

topics, probs = topic_model.fit_transform(df_negatif["clean_text"])

df_negatif["topic"] = topics

In [10]:
# 7. TOPIC INFO

topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,73,-1_alternatif_berkala_cont_channel,"[alternatif, berkala, cont, channel, solusi, p...",[transaksi melalui aplikasi mengalami masalah ...
1,0,2131,0_aplikasi_error_pakai_kalau,"[aplikasi, error, pakai, kalau, akun, mulu, ma...",[transfer rekening baru baca bagaimana error t...
2,1,80,1_aplikasi_wifi_close_silakan,"[aplikasi, wifi, close, silakan, cont, cache, ...",[mendatapati masalah silakan lakukan langkah b...
3,2,41,2_pahamjadiberkah_ikutan_icon_jawabannya,"[pahamjadiberkah, ikutan, icon, jawabannya, lo...",[waalaikumsalam minsy jawabannya ialah icon pa...
4,3,36,3_error_mluu_mint_enih,"[error, mluu, mint, enih, guys, info, begini, ...","[error, error, error]"
5,4,34,4_selalu_jaga_resmi_link,"[selalu, jaga, resmi, link, klik, jangan, pern...",[update sekarang versi appstore playstore moho...
6,5,31,5_pihak_memberikan_sandi_appstore,"[pihak, memberikan, sandi, appstore, playstore...",[sekarang versi terbaru appstore playstore moh...


In [11]:
# 8. PRINT KEYWORDS PER TOPIK

print("=== Keywords for All Topics ===")
for topic_id in topic_info["Topic"]:
    if topic_id == -1:   # skip outlier jika mau
        continue
    print(f"\nTOPIC {topic_id}")
    print(topic_model.get_topic(topic_id))

=== Keywords for All Topics ===

TOPIC 0
[('aplikasi', np.float64(0.05808842056716536)), ('error', np.float64(0.057139999896793255)), ('pakai', np.float64(0.03665130761125983)), ('kalau', np.float64(0.03374908350399949)), ('akun', np.float64(0.03251119096541)), ('mulu', np.float64(0.03176257942352001)), ('masuk', np.float64(0.030330647019172794)), ('terus', np.float64(0.02989660565495632)), ('hari', np.float64(0.02868868872357317)), ('buat', np.float64(0.026982182232805525))]

TOPIC 1
[('aplikasi', np.float64(0.09859132360148921)), ('wifi', np.float64(0.09349942815849317)), ('close', np.float64(0.0871288044701557)), ('silakan', np.float64(0.08620578049608216)), ('cont', np.float64(0.08107932729427543)), ('cache', np.float64(0.07565499047768165)), ('menggunakan', np.float64(0.07510188192732045)), ('restart', np.float64(0.07309767070329648)), ('versi', np.float64(0.07080116428373068)), ('overlapping', np.float64(0.06143310572745691))]

TOPIC 2
[('pahamjadiberkah', np.float64(0.4073475213